In [ ]:
import random
import time
from threading import Timer
import threading
import os
from pyautogui import press, typewrite, hotkey

#도시 매각 (현금이 부족하면 호출하면 됨) (총 재산이 통행료보다 적으면 호출하기 전에 파산 처리)
global default_map_name
default_map_name = ['start', '타이베이', '베이징', '마닐라', '제주', 'island', '아테네', '코펜하겐', '오타와', '상파울로', 'festival', '프라하', '베를린', '모스크바', '제네바', 'airport', '런던', '파리', '뉴욕', '건국대']
# 5.2.3.3. 매매가 배열
global trading_fee
trading_fee = [0, 500, 500, 500, 500, 0, 1000, 1000, 1000, 1000, 0, 1500, 1500, 1500, 1500, 0, 2000, 2000, 2000, 2000] 
# 현재 통행료 배열
global now_fee
now_fee = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# 6.1.4. 소유주 문자열 배열
global owner_list
owner_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

global landmark_list  
landmark_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

global now_order
now_order = 0

global islandPlayer
islandPlayer = [0,0,0,0]

global at_island

#로그인시 얻은 아이디 변수
global id_1
global id_2
global id_3
global id_4
id_1 = 'p1'
id_2 = 'p2'
id_3 = 'p3'
id_4 = 'p4'
global id_list

id_list = [id_1, id_2, id_3, id_4]

#6.1.1. 플레이어 수
global login_count
login_count = 4

#6.1.2 플레이어 정보
at_island = 0   #이건 로그에서 저장받아오기

global id_1_info
global id_2_info
global id_3_info
global id_4_info
global id_info_list
id_1_info = [id_1,10000,10000,0]
id_2_info = [id_2,10000,10000,0]
id_3_info = [id_3,10000,10000,0]
id_4_info = [id_4,10000,10000,0]

#id info 이차원 배열
id_info_list = [id_1_info,id_2_info,id_3_info,id_4_info]

is_double = True


def input_check(require, player_input):
   match player_input:
      case 'roll'|'ㅈㅅㅇ'|'주사위':
         if(require == 'roll'):        
            return True
         else:
            return False
      

def get_player_input(player_input_ref):
    global now_order
    global id_list
    
    player_input_ref[0] = input(id_list[now_order]+' > ')


        

#랜드마크는 인수가 불가능 하기 때문에 가능한 코드
#랜드마크 함수
def landMark(playerTurn):
    global landmark_list
    global owner_list
    for i in range(20):
        if owner_list[i] == playerTurn:
            landmark_list[i] = playerTurn
    print('===== 랜드마크 생성 완료! =====')
    for j in range(20):
        print('랜드마크 소유주: ' + str(landmark_list[j]))
        
        
#무인도 함수
def island(playerTurn):
    global islandPlayer
    islandPlayer[playerTurn - 1] = 1
    
#주사위 굴리기 함수, 두개의 주사위 합이 리턴된다, 3번째 더블 나올 시에 0이 리턴된다.
def rollDice(playerTurn): 
    
    firstDice = 0
    secondDice = 0
    global now_order
    global islandPlayer
    while(1):
        a = input('주사위를 굴리세요: ')
        if a == 'roll' or a == 'r' or a == '주사위' or a == 'ㅈㅅㅇ':
            firstDice = random.randrange(1,7) #첫번째 주사위
            secondDice = random.randrange(1,7) #두번째 주사위
            totalDice = firstDice + secondDice #주사위들의 합
            
            if islandPlayer[playerTurn - 1] == 1:
                print ('===== 첫번째 주사위 값: ' + str(firstDice) + ', 두번째 주사위 값: ' + str(secondDice) + ' =====')
                print ('===== 현재 무인도에 있습니다. 이동하지 않습니다. =====')
                islandPlayer[playerTurn - 1] = 0
                return 0
            
            if firstDice != secondDice:
                print('===== 첫번째 주사위 값: ' + str(firstDice) + ', 두번째 주사위 값: '+ str(secondDice) + ' =====')
                print('===== ' + str(totalDice) + '칸 이동합니다. =====')
                now_order = 0
                return totalDice
            else:
                print('===== 첫번째 주사위 값: ' + str(firstDice) + ', 두번째 주사위 값: '+ str(secondDice) + ' =====')
                now_order+=1
                if now_order==1:
                    print('===== 첫번째 더블! =====')
                    print('===== ' + str(totalDice) + '칸 이동합니다. =====')
                    return totalDice
                elif now_order==2:
                    print('===== 두번째 더블! =====')
                    print('===== ' + str(totalDice) + '칸 이동합니다. =====')
                    return totalDice
                elif now_order==3:
                    print('===== 세번째 더블! =====')
                    print('===== 무인도로 이동합니다 =====')
                    island(playerTurn)
                    return -1
        else:
            print('[Error]: 주사위 굴리기 명령어를 입력해주십시오')

    

#통행료가 현금보다 많을 경우 씀
#매각 함수
def sellProperty(playerTurn):
    global id_info_list
    global default_map_name
    global trading_fee
    global now_fee
    global owner_list
    while id_info_list[playerTurn-1][1] < now_fee[1]: #통행료를 낼 현금을 보유하고 있을 때 까지 한다
        print('매각할 도시를 입력해주세요.')
        cityName = input()
        try: #입력받은 인자가 정수 숫자일 경우 (도시번호)
            cityName = int(cityName) #입력 받은 것이 정수형인가?
            if owner_list[cityName] != playerTurn:
                print('보유하고 계신 땅이 아닙니다.')
                #특수지역 매각하려는 것도 보유하고 계신 땅이 아닙니다에 포함
                continue
            else:
                id_info_list[playerTurn-1][1] += trading_fee[cityName] #매각된 땅값을 현금에 포함
                owner_list[cityName] = 0 #소유지를 무소유로 바꿈
                print(default_map_name[cityName] + ' 도시 매각을 진행합니다.')
                print('통행료 내기 전 자산은: '+ str(id_info_list[playerTurn-1][1]))
                continue
        except: #숫자가 아닌 경우, 배열을 넘치는 숫자인 경우, int형이 아닌 경우
            try: #입력받은 인자가 숫자일 경우 (배열에 넘치는 숫자든 int형이 아니든)
                b = float(cityName) - 1 #이 식이 에러를 발생시키면 cityName은 문자형임
                print('[Error]: 유효하지 않은 도시번호입니다. 0이상 19이하의 정수를 입력해주세요.')
                continue
            except:
                a = 0
                for i in range(20):
                    if cityName == default_map_name[i]:
                        if owner_list[i] == playerTurn:
                            id_info_list[playerTurn-1][1] += trading_fee[i] #매각된 땅값을 현금에 포함
                            owner_list[i] = 0 #소유지를 무소유로 바꿈
                            print(default_map_name[i] + ' 도시 매각을 진행합니다.')
                            print('통행료 내기 전 자산은: '+ str(id_info_list[playerTurn-1][1]))
                            a = 1
                            break    
                        else:
                            print('보유하고 계신 땅이 아닙니다.')
                            a = 1
                            continue
                if a == 0:
                    print('[Error]: 유효하지 않은 도시명입니다. 유효한 도시명을 입력해주세요.')  

      
# 제한시간 내에 사용자 명령 받는 함수 (left_time : 제한시간)
def input_timer(left_time, require_msg):
    global player_input
    global now_order
    global id_list
    
    player_input=[None]
    input_timer= threading.Thread(target=get_player_input, args=(player_input,))
    input_timer.daemon = True

    input_timer.start()  
    for i in range(1, left_time):
        if input_check(require_msg,player_input[0]):
           return True
        elif player_input[0] != None:
           print('입력이 틀렸습니다.')
           player_input=[None]
           input_timer= threading.Thread(target=get_player_input, args=(player_input,))
           input_timer.daemon = True
           input_timer.start()

        time.sleep(1)
        
    if player_input[0] == None:
        press('enter')
        player_input=[None]
        print('===== %d초가 지났습니다. =====' % left_time)
        if(left_time == 10):
            print('===== 중도포기 하였습니다. =====')
            time.sleep(2)
            
            # 중도포기 함수 자리
            id_info_list[now_order][3] = 2
        else:
            print('===== 경고 1회 받았습니다. =====')
            time.sleep(2)
            # 경고 함수 자리
    else:
        
        time.sleep(2)




   

def main():
   #회원가입/로그인
   
   #인원

   #startgame

   #drawmap
   global now_order

   for i in range(10):
      for j in range(0, 4):

         #여기서부터가 플레이어의 턴, while문 내부가 차례
         while (at_island < 2 and id_info_list[j][3] < 2 and is_double):
            now_order = j
            valid_input = input_timer(10,'roll')

            


            if(valid_input):
               print('rolled dice')
               # 이동, 액션 함수 자리



               #해당 플레이어의 차례 종료
               break

               

               
               


main()